In [ ]:
!pip install -qU transformers peft trl accelerate bitsandbytes datasets

In [ ]:
#!pip install -qU flash-attn

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import json
import re
import os

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from huggingface_hub import notebook_login, HfFolder, HfApi
from datasets import load_dataset, DatasetDict, Dataset
# from dotenv import load_dotenv

import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from accelerate import Accelerator
import bitsandbytes as bnb
from trl import SFTTrainer
from peft import LoraConfig, AutoPeftModelForCausalLM
#from prompts import create_instruction, get_instruction_prompt_template

In [ ]:
if torch.cuda.is_available():
    print("GPU is available.")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"Total number of GPUs: {torch.cuda.device_count()}")
else:
    print("GPU is not available.")

GPU is available.
GPU Name: NVIDIA A100-SXM4-40GB
Total number of GPUs: 1


In [ ]:
HfFolder.save_token("token")
# notebook_login()

In [ ]:
# config = json.load(open('./config.json', 'r'))
# config
TASK = "cls_headline" # ["re_finred", "cls_headline", "sentiment_fbp_fiqasa","cls_fomc"]
VERSION = "v1"
MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct" # ["mistralai/Mistral-7B-Instruct-v0.2", "microsoft/Phi-3-mini-4k-instruct", "meta-llama/Meta-Llama-3-8B-Instruct"]
# MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2" # ["mistralai/Mistral-7B-Instruct-v0.2", "microsoft/Phi-3-mini-4k-instruct", "meta-llama/Meta-Llama-3-8B-Instruct"]
# MODEL_ID = "microsoft/Phi-3-mini-4k-instruct" # ["mistralai/Mistral-7B-Instruct-v0.2", "microsoft/Phi-3-mini-4k-instruct", "meta-llama/Meta-Llama-3-8B-Instruct"]
# DATASET_ID = "Sorour/headline-dir" # "Sorour/sentiment-train", "Sorour/headline"
DATASET_ID = "Sorour/headline-direction" # "Sorour/sentiment-train", "Sorour/headline-direction"

PADDING_SIDE = "right"

In [ ]:
if MODEL_ID=="mistralai/Mistral-7B-Instruct-v0.2":
    mdl= "mistral"
elif MODEL_ID=="microsoft/Phi-3-mini-4k-instruct":
    mdl= "phi3"
elif MODEL_ID=="meta-llama/Meta-Llama-3-8B-Instruct":
    mdl="llama3"
SAVE_MODEL_TO = f"Sorour/{mdl}_{TASK}"
OUTPUT_DIR = f"./{TASK}_{mdl}_{VERSION}"
print("SAVE_MODEL_TO: ", SAVE_MODEL_TO)
print("OUTPUT_DIR", OUTPUT_DIR)

SAVE_MODEL_TO:  Sorour/llama3_cls_headline
OUTPUT_DIR ./cls_headline_llama3_v1


In [ ]:
# model_id = config["model_id"]
# dataset_id = config["dataset_id"]

In [ ]:
df = load_dataset(DATASET_ID)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
df

DatasetDict({
    train: Dataset({
        features: ['text', 'instruction', 'label', 'type'],
        num_rows: 6493
    })
    valid: Dataset({
        features: ['text', 'instruction', 'label', 'type'],
        num_rows: 928
    })
    test: Dataset({
        features: ['text', 'instruction', 'label', 'type'],
        num_rows: 1856
    })
})

In [ ]:
# Load the model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
    #attn_implementation="flash_attention_2",
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, padding_side = PADDING_SIDE)
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
INSTRUCTION_PROMPT_TEMPLATE = """\
<|user|> {instruction}

sentence: {Context} <|end|><|assistant|>"""

RESPONSE_TEMPLATE = """\
{answer}<|end|>"""

In [ ]:
INSTRUCTION_PROMPT_TEMPLATE = """\
<s>[INST] <<SYS>> {instruction}<<SYS>>
Sentence: {Context}[/INST]"""

RESPONSE_TEMPLATE = """\
{answer}</s>"""

In [ ]:
INSTRUCTION_PROMPT_TEMPLATE = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{instruction}
<|eot_id|><|start_header_id|>user<|end_header_id|>

[SENT]{Context}[END_SENT]<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

RESPONSE_TEMPLATE = """\
{answer}<|eot_id|><|end_of_text|>"""

In [ ]:
def create_instruction(sample, return_response=True):
  prompt = INSTRUCTION_PROMPT_TEMPLATE.format(instruction=sample['instruction'],Context=sample['text'])

  if return_response:
    prompt += RESPONSE_TEMPLATE.format(answer=sample["label"])

  return prompt

In [ ]:
def get_inference_result(df, model_pipe, output_name, params):
# Create an empty list to store the outputs
    outputs_list = []

    for idx, row  in df.iterrows():
        print(idx)
        print(row['text'])
        result = model_pipe(create_instruction(row,  return_response=False), do_sample=params['do_sample'],
                            max_new_tokens=params['max_new_tokens'], temperature=params['temperature'], top_k=params['top_k'])
        outputs=result[0]["generated_text"]
        print(outputs)
        outputs_list.append(outputs)
    # Add the outputs_list as a new column to the DataFrame
    df[output_name] = outputs_list
    return df

In [ ]:
test_data = df['test']
df_test = pd.DataFrame(test_data)


In [ ]:
params = {
    "do_sample":False,
    "max_new_tokens":500,
    "temperature":0.1,
    "top_k":50
}
colname = 'predict_mistral'

# df_test_fiqasa = get_inference_result(df_test_fiqasa, ft_pipe, output_name = colname, params=params)
base_model_pipe = pipeline("text-generation", model, tokenizer=tokenizer, max_new_tokens=256, return_full_text=False)
df_test_headline = get_inference_result(df_test, base_model_pipe, output_name = colname, params=params)

In [ ]:
df_test_headline

In [ ]:
df_test_headline['label_mistral_e'] = df_test_headline['predict_mistral'].str.extract(r'(\b[up|down|stable|Yes|No|past|future]+\b)', flags=re.IGNORECASE)


In [ ]:
df_test_headline['label_phi3_e'] = df_test_headline['predict_phi3'].str.extract(r'(\b[Neutral|Dovish|Hawkish|]+\b)', flags=re.IGNORECASE)


In [ ]:
df_test_headline['predict_llama3'] = df_test_headline['predict_llama3'].str.replace('\n\n', '')
# df_test_headline['label_mistral_e'] = df_test_headline['label_mistral_e'].str.lower()
# df_test_headline['label'] = df_test_headline['label'].str.lower()

In [ ]:
import re
categories = [
    'founded by', 'chief executive officer', 'employer', 'product/material produced', 'industry', 'owned by', 'subsidiary',
    'parent organization', 'manufacturer', 'brand', 'owner of', 'developer', 'headquarters location', 'distribution format',
    'original broadcaster', 'legal form', 'location of formation', 'creator', 'stock exchange', 'operator', 'publisher',
    'distributed by', 'platform', 'member of', 'position held', 'currency', 'director/manager', 'chairperson', 'business division'
]

# Create a regex pattern to match any of the categories
pattern = r'\b(?:' + '|'.join(re.escape(category) for category in categories) + r')\b'

# Function to extract matching category from predict_llama3
def extract_category(text):
    match = re.search(pattern, text)
    return match.group(0) if match else None

# Apply the function to the dataframe
df_test_headline['extracted_category'] = df_test_headline['predict_llama3'].apply(extract_category)
df_test_headline['extracted_category'] = df_test_headline['extracted_category'].fillna('None')

In [ ]:
file_path='/content/drive/MyDrive/results-v2/finred_mistral_zero.csv'

df_test_headline.to_csv(file_path, index=False)

In [ ]:
def eval(y_pred, y_true):# Calculate evaluation params
    accuracy = accuracy_score(y_true, y_pred)

    # Calculate precision
    precision = precision_score(y_true, y_pred, average='weighted')

    # Calculate recall
    recall = recall_score(y_true, y_pred, average='weighted')

    # Calculate F1 score
    f1 = f1_score(y_true, y_pred, average='weighted')

    # Print the results
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    return (accuracy, precision, recall, f1)

In [ ]:
def eval_macro(y_pred, y_true):# Calculate evaluation params
    accuracy = accuracy_score(y_true, y_pred)

    # Calculate precision
    precision = precision_score(y_true, y_pred, average='macro')

    # Calculate recall
    recall = recall_score(y_true, y_pred, average='macro')

    # Calculate F1 score
    f1 = f1_score(y_true, y_pred, average='macro')

    # Print the results
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    return (accuracy, precision, recall, f1)

In [ ]:
df_test_headline['label'] = df_test_headline['label'].str.lower()
df_test_headline['label_mistral_e'] = df_test_headline['label_mistral_e'].str.lower()

In [ ]:
y_pred = df_test_headline['extracted_category']
y_true = df_test_headline['label']
(accuracy, precision, recall, f1) = eval(y_pred, y_true)

Accuracy: 0.17540322580645162
Precision: 0.3633800415807414
Recall: 0.17540322580645162
F1 Score: 0.2056457762839155


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
(accuracy, precision, recall, f1) = eval_macro(y_pred, y_true)

Accuracy: 0.17540322580645162
Precision: 0.3190370249042957
Recall: 0.20775901400823532
F1 Score: 0.20347810359191434


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", padding_side = PADDING_SIDE)
# tokenizer.pad_token = tokenizer.eos_token

In [ ]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM",
)

In [ ]:
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="steps",
    eval_steps=20,
    # max_steps=50,
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    save_strategy="epoch",
    learning_rate=2e-4,
    logging_steps=10,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    push_to_hub=True,
)

In [ ]:

max_seq_length=1024
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=df['train'],
    eval_dataset=df['valid'],
    # formatting_func= lambda sample: create_instruction(sample, return_response=True),
    formatting_func=create_instruction,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:440: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictionary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(


In [ ]:
import time
t0 = time.time
trainer.train()
t_run = time.time()
print(f"time to train : {t_run}")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
20,0.303800,0.296079
40,0.289900,0.280911
60,0.270700,0.271449
80,0.261500,0.269653
100,0.235700,0.270741
120,0.237700,0.266702
140,0.234600,0.266165
160,0.235700,0.261693


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_downlo

time to train : 1716422780.4437294


In [ ]:
# api = HfApi()
# api.create_repo(repo_id="cls_fomc_llama3_v1", token=HfFolder.get_token())

In [ ]:
trainer.save_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto"
)

merged_model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
merged_model.push_to_hub(SAVE_MODEL_TO)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Sorour/llama3_cls_headline/commit/2f7552c69a70c3d6d00a630921f510ed5e691726', commit_message='Upload LlamaForCausalLM', commit_description='', oid='2f7552c69a70c3d6d00a630921f510ed5e691726', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
ft_pipe = pipeline("text-generation", merged_model, tokenizer=tokenizer, max_new_tokens=256, return_full_text=False)

In [ ]:
# df_test_fiqasa = pd.DataFrame(load_dataset("Sorour/sentiment-test-fiqasa")['test'])
# df_test_fbp = pd.DataFrame(load_dataset("Sorour/sentiment-test-fbp")['test'])
df_test = pd.DataFrame(df['test'])
df_test

,text,instruction,label,type
0,dec gold down $2.40 at $374.50/oz after $373.5...,Analyze the provided data to identify the tren...,down,priceDirection
1,dec. gold falls 40 cents to $646.50/oz in morn...,Analyze the provided data to identify the tren...,down,priceDirection
2,"February gold slumped $9, or 0.7%, at $1,203/oz.",Analyze the provided data to identify the tren...,down,priceDirection
3,"Gold prices slide $14.90, or 1.1%, to $1,289.8...",Analyze the provided data to identify the tren...,down,priceDirection
4,"Gold jumps to Rs 32,350; silver rallies by Rs ...",Analyze the provided data to identify the tren...,up,priceDirection
...,...,...,...,...
1851,Gold little changed near 7-week low on weak te...,Analyze the provided data to identify the tren...,down,priceDirection
1852,"December gold rises 0.1% at $1,279.40/oz.",Analyze the provided data to identify the tren...,up,priceDirection
1853,Gold prices hold modest loss after PPI report,Analyze the provided data to identify the tren...,down,priceDirection
1854,"Gold prices log highest finish since mid-June,...",Analyze the provided data to identify the tren...,up,priceDirection


In [ ]:
def get_inference_result(df, model_pipe, output_name, params):
# Create an empty list to store the outputs
    outputs_list = []

    for idx, row  in df.iterrows():
        print(idx)
        print(row['text'])
        result = model_pipe(create_instruction(row, return_response=False), do_sample=params['do_sample'],
                            max_new_tokens=params['max_new_tokens'], temperature=params['temperature'], top_k=params['top_k'])
        outputs=result[0]["generated_text"]
        print(outputs)
        outputs_list.append(outputs)
    # Add the outputs_list as a new column to the DataFrame
    df[output_name] = outputs_list
    return df

In [ ]:
def eval(y_pred, y_true):# Calculate evaluation params
    accuracy = accuracy_score(y_true, y_pred)

    # Calculate precision
    precision = precision_score(y_true, y_pred, average='weighted')

    # Calculate recall
    recall = recall_score(y_true, y_pred, average='weighted')

    # Calculate F1 score
    f1 = f1_score(y_true, y_pred, average='weighted')

    # Print the results
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    return (accuracy, precision, recall, f1)

In [ ]:
def eval_macro(y_pred, y_true):# Calculate evaluation params
    accuracy = accuracy_score(y_true, y_pred)

    # Calculate precision
    precision = precision_score(y_true, y_pred, average='macro')

    # Calculate recall
    recall = recall_score(y_true, y_pred, average='macro')

    # Calculate F1 score
    f1 = f1_score(y_true, y_pred, average='macro')

    # Print the results
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    return (accuracy, precision, recall, f1)

In [ ]:
params = {
    "do_sample":False,
    "max_new_tokens":500,
    "temperature":0.1,
    "top_k":50
}
colname = 'predict'

# df_test_fiqasa = get_inference_result(df_test_fiqasa, ft_pipe, output_name = colname, params=params)

df_test= get_inference_result(df_test, ft_pipe, output_name = colname, params=params)

0
dec gold down $2.40 at $374.50/oz after $373.50 low
down
1
dec. gold falls 40 cents to $646.50/oz in morning trading


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


down
2
February gold slumped $9, or 0.7%, at $1,203/oz.
down
3
Gold prices slide $14.90, or 1.1%, to $1,289.80 an ounce
down
4
Gold jumps to Rs 32,350; silver rallies by Rs 250 per kg
up
5
gold down as inflation fear subsides; silver tanks
down
6
Gold futures end higher after two-session decline
up
7
spot gold trades lower


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Streaming output truncated to the last 5000 lines.
stable
190
Gold, silver sparkle on Diwali Muhurat trade
up
191
Gold at 1-week lows as dollar inches higher
down
192
gold futures fall 0.14 per cent on global cues
down
193
april gold up 60c at $337.20, off $343.50 high
up
194
Gold futures rise to Rs 30,875; silver spurts Rs 187 per kg
up
195
Gold holds losses after jobs report
down
196
Gold prices log highest settlement in two-and-a-half weeks
up
197
barclays expects gold to hit new low as fed plans rate hike
down
198
Gold rises on positive global cues
up
199
Gold edges higher with Fed rate outlook in focus
up
200
gold dips, dollar gains as markets pause after two-day rally
down
201
gold futures end lower for the day, up for the week
up
202
Gold goes up 0.11% on positive global cues
up
203
Gold books biggest one-day loss of 2017
down
204
mcx gold prices to trade sideways: angel commodities
stable
205
Profit-booking pulls gold futures down to Rs 29,781
down
206
gold trims gains; oil add

In [ ]:
df_test.head()

,text,instruction,label,type,predict
0,dec gold down $2.40 at $374.50/oz after $373.5...,Analyze the provided data to identify the tren...,down,priceDirection,down
1,dec. gold falls 40 cents to $646.50/oz in morn...,Analyze the provided data to identify the tren...,down,priceDirection,down
2,"February gold slumped $9, or 0.7%, at $1,203/oz.",Analyze the provided data to identify the tren...,down,priceDirection,down
3,"Gold prices slide $14.90, or 1.1%, to $1,289.8...",Analyze the provided data to identify the tren...,down,priceDirection,down
4,"Gold jumps to Rs 32,350; silver rallies by Rs ...",Analyze the provided data to identify the tren...,up,priceDirection,up


In [ ]:
df_test['predict'] = df_test['predict'].str.replace(r'^ ', '', regex=True)

In [ ]:
df_test['predict'] = df_test['predict'].str.replace(' ', '')

In [ ]:
file_path='/content/drive/MyDrive/results-v3/finred_llama3_ft.csv'

df_test.to_csv(file_path, index=False)

In [ ]:
y_pred = df_test['predict']
y_true = df_test['label']
(accuracy, precision, recall, f1) = eval(y_pred, y_true)

Accuracy: 0.9558189655172413
Precision: 0.9555606396408899
Recall: 0.9558189655172413
F1 Score: 0.9553801068342163


In [ ]:
(accuracy, precision, recall, f1) = eval_macro(y_pred, y_true)

Accuracy: 0.7459677419354839
Precision: 0.7559550069979
Recall: 0.7233594447769057
F1 Score: 0.7124708134668448


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
!zip -r /content/cls_headline_llama3_v1.zip /content/cls_headline_llama3_v1
#!cp source destination

  adding: content/cls_headline_llama3_v1/ (stored 0%)
  adding: content/cls_headline_llama3_v1/README.md (deflated 52%)
  adding: content/cls_headline_llama3_v1/runs/ (stored 0%)
  adding: content/cls_headline_llama3_v1/runs/May19_06-17-43_802e93aef622/ (stored 0%)
  adding: content/cls_headline_llama3_v1/runs/May19_06-17-43_802e93aef622/events.out.tfevents.1716099470.802e93aef622.1326.0 (deflated 62%)
  adding: content/cls_headline_llama3_v1/adapter_model.safetensors (deflated 7%)
  adding: content/cls_headline_llama3_v1/special_tokens_map.json (deflated 63%)
  adding: content/cls_headline_llama3_v1/tokenizer.json (deflated 74%)
  adding: content/cls_headline_llama3_v1/adapter_config.json (deflated 53%)
  adding: content/cls_headline_llama3_v1/tokenizer_config.json (deflated 96%)
  adding: content/cls_headline_llama3_v1/checkpoint-66/ (stored 0%)
  adding: content/cls_headline_llama3_v1/checkpoint-66/README.md (deflated 66%)
  adding: content/cls_headline_llama3_v1/checkpoint-66/adapt

In [ ]:
# echo "export HF_HOME='your_huggingface_token_here'" >> ~/.bashrc
# source ~/.bashrc

# echo "export HF_HOME='your_huggingface_token_here'" >> ~/.zshrc
# source ~/.zshrc

# unset HF_HOME